<a href="https://colab.research.google.com/github/MRdvan/cs231n_Assignments/blob/master/CIFAR_10_with_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = "/assignment2_colab/assignment2"
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# This downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
!bash get_datasets.sh
%cd /content/drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/assignment2_colab/assignment2/cs231n/datasets
/content/drive/My Drive/assignment2_colab/assignment2


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T
import torch.nn.functional as F 
import numpy as np

USE_GPU = True
dtype = torch.float32 # We will be using float throughout this tutorial.

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss.
print_every = 100
print('using device:', device)

using device: cpu


In [ ]:
NUM_TRAIN = 49000

#uygulanacak preprocessing yöntemlerinin eklendiği modul, 
#burda data augmentation, normalization, gibi işlemler yapılabilir. 

#Data Augmentation : merkezin kırpması,y ekseninde ters çevirme,blurlama eklenerek veri arttırımı yapıldı
transform = T.Compose([
                T.ToTensor(),
                #T.RandomVerticalFlip(0.5),
                #T.RandomRotation(5),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

#datasetin preprocessing uygulanarak yüklenmesi ve dataloader ile mini-batch olarak alınması.
#ve datasetin train,val ve test olacak şekilde üç parçaya ayrılması.
cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./cs231n/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))


def train_part34(model, optimizer,scheduler ,epochs=1):

    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            optimizer.zero_grad()

            loss.backward()
            
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()
        scheduler.step()

In [ ]:
import torch
import torch.nn as nn

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    """iki tane conv layerının residual connection ile beraber kullanılarak oluşturulan block
       dilation : bottleneck ile boyut arttırılıcağını gösterir.
    """
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.dropout1 = nn.Dropout(0.25)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.dropout2 = nn.Dropout(0.3)
        self.downsample = downsample
        self.stride = stride
        

    def forward(self, x):
        identity = x #residual bağlantı için tutuldu

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.dropout2(out)

        if self.downsample is not None: #64->128 gibi bir ara geçiş yapılıcak ise
            identity = self.downsample(x) #inputun boyutu arttırıldı

        out += identity# z=(w.x+b)+identity
        out = self.relu(out)# g(z)

        return out


In [ ]:
class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)

    expansion = 4 #boyut arttırma katsayısı

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.dropout = self.nn.Dropout(0.3)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x #

        out = self.conv1(x)#1x1 conv boyut düşürüldü
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)#3x3 conv 
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv3(out)#1x1 conv boyut arttı 
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity #residual connection eklendi (boyut düşürülüp, arttırılıp eski haline getirildiği için eklenebildi)
        out = self.relu(out)

        return out

In [ ]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=10, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,dilate=replace_stride_with_dilation[2])
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        
        #weights initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)


def _resnet(arch, block, layers, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    return model


def resnet18(pretrained=False, progress=True, **kwargs):
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], progress,
                   **kwargs)
    
def resnet34(pretrained=False, progress=True, **kwargs):
    return _resnet('resnet34', BasicBlock, [3, 4, 6, 3], progress,
                   **kwargs)
    
def resnet50(pretrained=False, progress=True, **kwargs):
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], progress,
                   **kwargs)

In [ ]:
################################################################################
# TODO:                                                                        #         
# Experiment with any architectures, optimizers, and hyperparameters.          #
# Achieve AT LEAST 70% accuracy on the *validation set* within 10 epochs.      #
#                                                                              #
# Note that you can use the check_accuracy function to evaluate on either      #
# the test set or the validation set, by passing either loader_test or         #
# loader_val as the second argument to check_accuracy. You should not touch    #
# the test set until you have finished your architecture and  hyperparameter   #
# tuning, and only run the test set once at the end to report a final value.   #
################################################################################
model = None
optimizer = None

# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


#optimizer = optim.SGD(model.parameters(), lr=learning_rate,momentum=0.9, nesterov=True)
#scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

learning_rate = 1e-3
model = resnet34()
optimizer = optim.Adam(model.parameters(),lr=learning_rate, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)
pass

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

# You should get at least 70% accuracy
train_part34(model, optimizer,scheduler, epochs=10)

Iteration 0, loss = 2.7578
Checking accuracy on validation set
Got 77 / 1000 correct (7.70)



KeyboardInterrupt: ignored

In [ ]:
##--- model 1 accuracy ---## 
best_model = model
check_accuracy_part34(loader_test, best_model)
check_accuracy_part34(loader_val, best_model)
check_accuracy_part34(loader_train, best_model)

Checking accuracy on test set
Got 7903 / 10000 correct (79.03)
Checking accuracy on validation set
Got 793 / 1000 correct (79.30)
Checking accuracy on validation set
Got 46318 / 49000 correct (94.53)


In [ ]:
##--- model 1 accuracy ---## 
best_model = model
check_accuracy_part34(loader_test, best_model)
check_accuracy_part34(loader_val, best_model)
check_accuracy_part34(loader_train, best_model)

Checking accuracy on test set
Got 7281 / 10000 correct (72.81)
Checking accuracy on validation set
Got 742 / 1000 correct (74.20)
Checking accuracy on validation set
Got 38811 / 49000 correct (79.21)


In [ ]:
##--- model 1 accuracy ---## 
best_model = model
check_accuracy_part34(loader_test, best_model)
check_accuracy_part34(loader_val, best_model)
check_accuracy_part34(loader_train, best_model)

Checking accuracy on test set
Got 7306 / 10000 correct (73.06)
Checking accuracy on validation set
Got 731 / 1000 correct (73.10)
Checking accuracy on validation set
Got 39083 / 49000 correct (79.76)


In [ ]:
##--- model 1 accuracy ---## 
best_model = model
check_accuracy_part34(loader_test, best_model)
check_accuracy_part34(loader_val, best_model)
check_accuracy_part34(loader_train, best_model)

Checking accuracy on test set
Got 7858 / 10000 correct (78.58)
Checking accuracy on validation set
Got 795 / 1000 correct (79.50)
Checking accuracy on validation set
Got 42443 / 49000 correct (86.62)


In [ ]:
##--- model 1 accuracy ---## 
best_model = model
check_accuracy_part34(loader_test, best_model)
check_accuracy_part34(loader_val, best_model)
check_accuracy_part34(loader_train, best_model)

Checking accuracy on test set
Got 7472 / 10000 correct (74.72)
Checking accuracy on validation set
Got 746 / 1000 correct (74.60)
Checking accuracy on validation set
Got 39057 / 49000 correct (79.71)


In [ ]:
##--- model 1 accuracy ---## 
best_model = model
check_accuracy_part34(loader_test, best_model)
check_accuracy_part34(loader_val, best_model)
check_accuracy_part34(loader_train, best_model)

Checking accuracy on test set
Got 6718 / 10000 correct (67.18)
Checking accuracy on validation set
Got 692 / 1000 correct (69.20)
Checking accuracy on validation set
Got 33986 / 49000 correct (69.36)
